In [ ]:
from scaling_function import *
from scipy.integrate import quad

In [ ]:
d = 3
d_t = 5

# Scaling Function

In [ ]:
sf = PrimalScalingFunction(d)
sf.plot()

- Urban - 2009, Figure 2.11-2.14

In [ ]:
l1_norm = quad(sf, sf.l1, sf.l2)[0]
print("L1 norm =", l1_norm)
assert np.isclose(l1_norm, 1.)

l2_norm = np.sqrt(quad(lambda x: sf(x)**2, sf.l1, sf.l2)[0])
print("L2 norm =", l2_norm)

In [ ]:
# L2 normalization
sf.plot(scale=1./l2_norm)

- Urban - 2009, Figure 5.5-5.11

In [ ]:
# partition of unity
nx = 200
x = np.linspace(-d, d, nx)
s = np.zeros(nx)
for k in range(sf.l1, sf.l2 + 1):
    s += sf(x + k)
plt.plot(x, s)
plt.show()
assert np.isclose(s[nx//2], 1.)

In [ ]:
# first order derivative
sf.plot(nu=1)

sf_deriv = sf.derivative()
h1_norm = np.sqrt(quad(lambda x: sf_deriv(x)**2, sf.l1, sf.l2)[0] + l2_norm**2)
print("H1 norm =", h1_norm)

In [ ]:
a = sf.refinement_coeffs()
sum_a = a.sum()
print("refinement coefficients =", a)
print("sum =", sum_a)
assert np.isclose(sum_a, 2.)

- Urban - 2009, Table 2.6-2.7

In [ ]:
g = sf.gramian()
if d == 3:
    g_ref = np.array([1/120, 13/60, 11/20, 13/60, 1/120])
    assert np.allclose(g, g_ref)

l2_norm_g = np.sqrt(g[d-1])
print("L2 norm from gramian =", l2_norm_g)
print("L2 norm from quadrature =", l2_norm)
assert np.isclose(l2_norm_g, l2_norm)

- Primbs - 2008: For $d = 3$, $g = [1/120, 13/60, 11/20, 13/60, 1/120]$

In [ ]:
h = sf.inner_product(nu=1)
if d == 3:
    h_ref = np.array([-32/3, -64/3, 64, -64/3, -32/3]) / 2**6
    assert np.allclose(h, h_ref)

h1_norm_h = np.sqrt(h[d-1] + g[d-1])
print("H1 norm from inner product =", h1_norm_h)
print("H1 norm from quadrature =", h1_norm)
assert np.isclose(h1_norm_h, h1_norm)

- Primbs - 2006, Beispiel 3.26: For $d = 3$ and $j = 3$, $h = [-32/3, -64/3, 64, -64/3, -32/3]$

# Dual Scaling Function

In [ ]:
sf_t = DualScalingFunction(d, d_t)
sf_t.plot()

- Urban - 2009, Figure 2.11-2.14

In [ ]:
l1_norm = quad(sf_t, sf_t.l1, sf_t.l2)[0]
print("L1 norm =", l1_norm)
assert np.isclose(l1_norm, 1., atol=1e-3)

l2_norm = np.sqrt(quad(lambda x: sf_t(x)**2, sf_t.l1, sf_t.l2)[0])
print("L2 norm =", l2_norm)

In [ ]:
# L2 normalization
sf_t.plot(scale=1./l2_norm)

- Urban - 2009, Figure 5.5-5.11

In [ ]:
a = sf_t.refinement_coeffs()
sum_a = a.sum()
print("refinement coefficients =", a)
print("sum =", sum_a)
assert np.isclose(sum_a, 2.)

- Urban - 2009, Table 2.6-2.7

In [ ]:
# partition of unity
nx = 200
x = np.linspace(-d-2*(d_t-1), d+2*(d_t-1), nx)
s = np.zeros(nx)
for k in range(sf_t.l1, sf_t.l2 + 1):
    s += sf_t(x + k)
plt.plot(x, s)
plt.show()
assert np.isclose(s[nx//2], 1.)

In [ ]:
# biorthogonality
res = []
for k in range(sf.l1-sf_t.l2+1, sf.l2-sf_t.l1):
    res.append(quad(lambda x: sf(x)*sf_t(x-k), sf.l1, sf.l2)[0])
res = np.array(res)
print("biorthogonality:")
print(res)

exact = np.zeros_like(res)
exact[res.size//2] = 1.
assert np.allclose(res, exact, atol=1e-3)

# Mother Wavelet

In [ ]:
mw = MotherWavelet(d, d_t)
mw.plot()

In [ ]:
l1_norm = quad(mw, mw.l1, mw.l2)[0]
print("L1 norm =", l1_norm)
assert np.isclose(l1_norm, 0.)

l2_norm = np.sqrt(quad(lambda x: mw(x)**2, mw.l1, mw.l2)[0])
print("L2 norm =", l2_norm)

In [ ]:
# L2 normalization
mw.plot(scale=1./l2_norm)

- Urban - 2009, Figure 5.5-5.11 (unknown normalization)

In [ ]:
# first order derivative
mw.plot(nu=1)

mw_deriv = mw.derivative()
h1_norm = np.sqrt(quad(lambda x: mw_deriv(x)**2, mw.l1, mw.l2)[0] + l2_norm**2)
print("H1 norm =", h1_norm)

In [ ]:
g = mw.gramian()
l2_norm_g = np.sqrt(g[d+d_t-2])
print("L2 norm from gramian =", l2_norm_g)
print("L2 norm from quadrature =", l2_norm)
assert np.isclose(l2_norm_g, l2_norm)

In [ ]:
h = mw.inner_product(nu=1)
h1_norm_h = np.sqrt(h[d+d_t-2] + g[d+d_t-2])
print("H1 norm from inner product =", h1_norm_h)
print("H1 norm from quadrature =", h1_norm)
assert np.isclose(h1_norm_h, h1_norm)